In [1]:
import pandas as pd

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset = pd.get_dummies(dataset,dtype=int, drop_first=True)

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [6]:
dataset=dataset.drop("User ID",axis=1)

In [7]:
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [8]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [9]:
dataset.columns

Index(['Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [10]:
independent=dataset[['Gender_Male','Age','EstimatedSalary']]

In [11]:
independent.shape

(400, 3)

In [12]:
independent

,Gender_Male,Age,EstimatedSalary
0,1,19,19000
1,1,35,20000
2,0,26,43000
3,0,27,57000
4,1,19,76000
...,...,...,...
395,0,46,41000
396,1,51,23000
397,0,50,20000
398,1,36,33000


In [13]:
dependent=dataset[[ 'Purchased',]]

In [14]:
from sklearn.model_selection import train_test_split    
x_train,x_test,y_train,y_test=train_test_split(independent,dependent,test_size=0.30,random_state=0)

In [15]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)


In [16]:
from sklearn.neighbors import KNeighborsClassifier

In [17]:
from sklearn.model_selection import GridSearchCV               
param_grid = {'weights':['uniform', 'distance'],
              'algorithm':['auto', 'ball_tree', 'kd_tree', 'brute']}
grid = GridSearchCV(KNeighborsClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted') 
grid.fit(x_train,y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


C:\sabeen\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


GridSearchCV(estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
                         'weights': ['uniform', 'distance']},
             scoring='f1_weighted', verbose=3)

In [18]:
re=grid.cv_results_

grid_prediction=grid.predict(x_test)



In [19]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,grid_prediction)
print(cm)

[[73  6]
 [ 4 37]]


In [20]:
from sklearn.metrics import classification_report
clf_report=classification_report(y_test,grid_prediction)
print(clf_report)

              precision    recall  f1-score   support

           0       0.95      0.92      0.94        79
           1       0.86      0.90      0.88        41

    accuracy                           0.92       120
   macro avg       0.90      0.91      0.91       120
weighted avg       0.92      0.92      0.92       120



In [21]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_prediction,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)


The f1_macro value for best parameter {'algorithm': 'auto', 'weights': 'uniform'}: 0.9171245421245421


In [22]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[73  6]
 [ 4 37]]


In [23]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.95      0.92      0.94        79
           1       0.86      0.90      0.88        41

    accuracy                           0.92       120
   macro avg       0.90      0.91      0.91       120
weighted avg       0.92      0.92      0.92       120



In [24]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])


0.9377894411855511

In [25]:
table=pd.DataFrame.from_dict(re)
table


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_algorithm,param_weights,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.009933,0.012101,0.015070,0.004376,auto,uniform,"{'algorithm': 'auto', 'weights': 'uniform'}",0.874254,0.874254,0.876643,0.929144,0.946153,0.900089,0.031147,1
1,0.008447,0.003744,0.009573,0.001280,auto,distance,"{'algorithm': 'auto', 'weights': 'distance'}",0.855314,0.855314,0.859435,0.929144,0.946153,0.889072,0.040054,5
2,0.001001,0.002002,0.013977,0.005753,ball_tree,uniform,"{'algorithm': 'ball_tree', 'weights': 'uniform'}",0.874254,0.874254,0.876643,0.929144,0.946153,0.900089,0.031147,1
3,0.006482,0.003726,0.007671,0.002947,ball_tree,distance,"{'algorithm': 'ball_tree', 'weights': 'distance'}",0.855314,0.855314,0.859435,0.929144,0.946153,0.889072,0.040054,5
4,0.006104,0.005500,0.012049,0.005164,kd_tree,uniform,"{'algorithm': 'kd_tree', 'weights': 'uniform'}",0.874254,0.874254,0.876643,0.929144,0.946153,0.900089,0.031147,1
5,0.001531,0.000535,0.008176,0.001365,kd_tree,distance,"{'algorithm': 'kd_tree', 'weights': 'distance'}",0.855314,0.855314,0.859435,0.929144,0.946153,0.889072,0.040054,5
6,0.000201,0.000402,0.118671,0.005952,brute,uniform,"{'algorithm': 'brute', 'weights': 'uniform'}",0.874254,0.874254,0.876643,0.929144,0.946153,0.900089,0.031147,1
7,0.000272,0.000543,0.058677,0.044595,brute,distance,"{'algorithm': 'brute', 'weights': 'distance'}",0.855314,0.855314,0.859435,0.929144,0.946153,0.889072,0.040054,5
